O input de dados para uma função de janela é uma ou mais linhas de um SELECT. 

SQLite possui as seguintes funções:

    - row_number()
    - rank()
    - dense_rank()
    - percent_rank()
    - cume_dist()
    - ntile()
    - lag(expr), lag(expr, offset), lag(expr, offset, default)
    - lead(expr), lead(expr, offset), lead(expr, offset, default)
    - first_value(expr)
    - last_value(expr)
    - nth_value(expr, n)

Diferente das funções de agregação que retornavam apenas um valor por grupo, as funções de janela retornam um valor para cada linha do grupo.


In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('../primeiro_banco')
cur = con.cursor()
print('conexão aberta')

conexão aberta


In [3]:
query = '''
SELECT * FROM Customers LIMIT 5;
'''

df = pd.read_sql_query(query, con)
df

,CustomerKey,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
0,11000,JON,YANG,4/8/1966,M,M,jon24@adventure-works.com,90000.0,2,Bachelors,Professional,Y
1,11001,EUGENE,HUANG,5/14/1965,S,M,eugene10@adventure-works.com,60000.0,3,Bachelors,Professional,N
2,11002,RUBEN,TORRES,8/12/1965,M,M,ruben35@adventure-works.com,60000.0,3,Bachelors,Professional,Y
3,11003,CHRISTY,ZHU,2/15/1968,S,F,christy12@adventure-works.com,70000.0,0,Bachelors,Professional,N
4,11004,ELIZABETH,JOHNSON,8/8/1968,S,F,elizabeth5@adventure-works.com,80000.0,5,Bachelors,Professional,Y


Função de janela com funções de agregação (SUM, AVG, MIN, MAX, COUNT)

In [5]:
query = '''
SELECT FirstName, MaritalStatus, AnnualIncome,
ROUND(AVG(AnnualIncome) OVER (PARTITION BY MaritalStatus), 2) AS Media
FROM Customers LIMIT 15;
'''
#PARTITION BY MaritalStatus tem a mesma função do GROUP BY
df = pd.read_sql_query(query, con)
df

,FirstName,MaritalStatus,AnnualIncome,Media
0,JON,M,90000.0,60313.74
1,RUBEN,M,60000.0,60313.74
2,MARCO,M,60000.0,60313.74
3,CURTIS,M,60000.0,60313.74
4,LAUREN,M,100000.0,60313.74
5,IAN,M,100000.0,60313.74
6,WYATT,M,30000.0,60313.74
7,ETHAN,M,40000.0,60313.74
8,SETH,M,40000.0,60313.74
9,RUSSELL,M,60000.0,60313.74


In [20]:
query = '''
SELECT CustomerKey, FirstName, MaritalStatus, AnnualIncome,
SUM(AnnualIncome) OVER (ORDER BY CustomerKey) AS Total,
SUM(AnnualIncome) OVER() AS TotalGeral,
ROUND(SUM(AnnualIncome) OVER (ORDER BY CustomerKey) * 100 / SUM(AnnualIncome) OVER(), 2) AS Percentual
FROM Customers LIMIT 10;
'''
#PARTITION BY MaritalStatus tem a mesma função do GROUP BY
df = pd.read_sql_query(query, con)
df

,CustomerKey,FirstName,MaritalStatus,AnnualIncome,Total,TotalGeral,Percentual
0,11000,JON,M,90000.0,90000.0,1.039320e+09,0.01
1,11001,EUGENE,S,60000.0,150000.0,1.039320e+09,0.01
2,11002,RUBEN,M,60000.0,210000.0,1.039320e+09,0.02
3,11003,CHRISTY,S,70000.0,280000.0,1.039320e+09,0.03
4,11004,ELIZABETH,S,80000.0,360000.0,1.039320e+09,0.03
5,11005,JULIO,S,70000.0,430000.0,1.039320e+09,0.04
6,11007,MARCO,M,60000.0,490000.0,1.039320e+09,0.05
7,11008,ROBIN,S,60000.0,550000.0,1.039320e+09,0.05
8,11009,SHANNON,S,70000.0,620000.0,1.039320e+09,0.06
9,11010,JACQUELYN,S,70000.0,690000.0,1.039320e+09,0.07


Funções ROW_NUMBER, RANK, DENSE_RANK

In [22]:
#ROW_NUMBER

query = '''
SELECT CustomerKey, FirstName, MaritalStatus, AnnualIncome,
    ROW_NUMBER() OVER (ORDER BY CustomerKey) AS RowNumber
FROM Customers
ORDER BY RowNumber
LIMIT 10;
'''

df = pd.read_sql_query(query, con)
df

,CustomerKey,FirstName,MaritalStatus,AnnualIncome,RowNumber
0,11000,JON,M,90000.0,1
1,11001,EUGENE,S,60000.0,2
2,11002,RUBEN,M,60000.0,3
3,11003,CHRISTY,S,70000.0,4
4,11004,ELIZABETH,S,80000.0,5
5,11005,JULIO,S,70000.0,6
6,11007,MARCO,M,60000.0,7
7,11008,ROBIN,S,60000.0,8
8,11009,SHANNON,S,70000.0,9
9,11010,JACQUELYN,S,70000.0,10


In [23]:
#ROW_NUMBER sobre PARTITION

query = '''
SELECT CustomerKey, FirstName, MaritalStatus, AnnualIncome,
    ROW_NUMBER() OVER (PARTITION BY MaritalStatus ORDER BY CustomerKey) AS RowNumber
FROM Customers
ORDER BY RowNumber
LIMIT 10;
'''

df = pd.read_sql_query(query, con)
df

,CustomerKey,FirstName,MaritalStatus,AnnualIncome,RowNumber
0,11000,JON,M,90000.0,1
1,11001,EUGENE,S,60000.0,1
2,11002,RUBEN,M,60000.0,2
3,11003,CHRISTY,S,70000.0,2
4,11007,MARCO,M,60000.0,3
5,11004,ELIZABETH,S,80000.0,3
6,11011,CURTIS,M,60000.0,4
7,11005,JULIO,S,70000.0,4
8,11012,LAUREN,M,100000.0,5
9,11008,ROBIN,S,60000.0,5


Função NTILE

In [36]:
#NTILE

query = '''
SELECT CustomerKey, FirstName, MaritalStatus, AnnualIncome,
    NTILE(100) OVER (ORDER BY AnnualIncome) AS Percentil
FROM Customers
ORDER BY Percentil
LIMIT 5000;
'''
#Existem as variações DECILES, QUARTILES
df = pd.read_sql_query(query, con)
df

,CustomerKey,FirstName,MaritalStatus,AnnualIncome,Percentil
0,11025,ALEJANDRO,M,10000.0,1
1,11030,BETHANY,M,10000.0,1
2,11035,WENDY,M,10000.0,1
3,11038,DIANA,M,10000.0,1
4,11126,SHAUN,M,10000.0,1
...,...,...,...,...,...
4995,28507,DEANNA,M,30000.0,28
4996,28508,JAIME,M,30000.0,28
4997,28509,AUDREY,S,30000.0,28
4998,28510,GREGORY,S,30000.0,28


LAG e LEAD

In [45]:
#LAG
query = '''
SELECT *,
LAG(Timestamp, 1) OVER (ORDER BY Timestamp) AS TimeStampAnterior
FROM Temperature 
ORDER BY Timestamp
LIMIT 5;

'''

df = pd.read_sql_query(query, con)
df

,Timestamp,Temperature,TimeStampAnterior
0,2021-08-21 11:34:50.711019,18.916893,None
1,2021-08-22 11:34:50.711019,20.689565,2021-08-21 11:34:50.711019
2,2021-08-23 11:34:50.711019,23.393948,2021-08-22 11:34:50.711019
3,2021-08-24 11:34:50.711019,25.135592,2021-08-23 11:34:50.711019
4,2021-08-25 11:34:50.711019,23.200781,2021-08-24 11:34:50.711019


In [44]:
#LEAD
query = '''
SELECT *,
LEAD(Timestamp, 2) OVER (ORDER BY Timestamp) AS TimeStampAnterior
FROM Temperature 
ORDER BY Timestamp
LIMIT 5000;

'''

df = pd.read_sql_query(query, con)
df

,Timestamp,Temperature,TimeStampAnterior
0,2021-08-21 11:34:50.711019,18.916893,2021-08-23 11:34:50.711019
1,2021-08-22 11:34:50.711019,20.689565,2021-08-24 11:34:50.711019
2,2021-08-23 11:34:50.711019,23.393948,2021-08-25 11:34:50.711019
3,2021-08-24 11:34:50.711019,25.135592,2021-08-26 11:34:50.711019
4,2021-08-25 11:34:50.711019,23.200781,2021-08-27 11:34:50.711019
...,...,...,...
495,2022-12-29 11:34:50.711019,19.783678,2022-12-31 11:34:50.711019
496,2022-12-30 11:34:50.711019,24.902536,2023-01-01 11:34:50.711019
497,2022-12-31 11:34:50.711019,19.372404,2023-01-02 11:34:50.711019
498,2023-01-01 11:34:50.711019,20.049503,None
